## Machine Learning Competition Notebook Template

### Student Name : 

**Name Surname ID**  


#### Summary
Please write here the summary of your implementation(s).

## load datasets

In [1]:
import pandas as pd
import sklearn as sk
import plotly as pl
import matplotlib
import matplotlib.pyplot as plt
import math
import patsy as patsy
import pickle
import statsmodels.api as sm
import numpy as np 
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#import functions as f

dftrain = pd.read_csv("/Users/andrewtobin/College/final_year/FYP/House_pricing_dataset-master/dataset_csv/train.csv",index_col='ad_id')
dftest  = pd.read_csv( "/Users/andrewtobin/College/final_year/FYP/House_pricing_dataset-master/dataset_csv/test.csv",index_col='ad_id')

In [2]:
train_data, test_data = train_test_split(dftrain, test_size=0.2)

print('Train Data Rows: ',len(train_data))

print('Test Data Rows: ',len(test_data))

Train Data Rows:  2385
Test Data Rows:  597


In [2]:
# data visualisation

## Data Cleaning, Features selection and preprocessing

In [3]:
def split_facility(df):
    split_df = df['facility'].str.split(',', expand = True) # splits the unstructured lists
    facility_dummy = pd.get_dummies(split_df[[0,1,2,3,4]].apply(pd.Series), prefix = '') # creates dummies from unstructured lists (duplicated columns)

    def sjoin(x): return ';'.join(x[x.notnull()].astype(str)) 
    facility_dummy = facility_dummy.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1)) ## joins columns but keeps multiple values i.e. 0;0;0;0

    for column in facility_dummy: ## workaround such that only one value is kept 
        facility_dummy[column] = facility_dummy[column].str.replace(";", "") ## removes the separator ';'
        facility_dummy[column] = facility_dummy[column].astype('int')

    facility_dummy[facility_dummy >= 1] = 1
    return facility_dummy

In [4]:

def prep_data(df):
    ## drop unnecessary columns
    df = df.drop(['county', 'environment', 'description_block', 'features'], axis=1)

    ## correct ber exempt value in ber_classification column
    df['ber_classification'] = df['ber_classification'].str.replace('SINo666of2006exempt', 'BER Exempt')

    ## create a log transform column for price to cut down the variance
    df['log_price'] = df.price.apply(lambda x: math.log(x))

    ## dropping rows where price/beds/baths is null for now for now (we might look at imputing these missing values later)
    df = df[df['price'].notnull()]
    df =df[df['beds'].notnull()]
    df =df[df['beds'].notnull()]
    return df

In [5]:
def get_dummies(df):
    area_dummy = pd.get_dummies(df['area'], prefix='area')
    ber_dummy = pd.get_dummies(df['ber_classification'], prefix='ber')
    property_type_dummy = pd.get_dummies(df['property_type'], prefix='property_type')

    dummy_matrix = pd.concat([area_dummy, ber_dummy, property_type_dummy], axis=1)

    return dummy_matrix

In [6]:
def make_predictions(df, x, model):
    ## makes a prediction for the log price
    df['log_predicted_price'] = model.predict(x.astype(float))
    

    ## gets the exponent of the log price to arrive at final predicted price
    df['predicted_price'] = np.exp(df['log_predicted_price'])

In [7]:
train_data = prep_data(train_data)
test_data = prep_data(test_data)

In [8]:
## put together full dummy matrix for all categorical variables
train_dummy = get_dummies(train_data)
fac_dummy = split_facility(train_data)
train_dummy = pd.concat([train_dummy, fac_dummy], axis=1)

## do the same for test data
test_dummy = get_dummies(test_data)
fac_dummy2 = split_facility(test_data)
test_dummy = pd.concat([test_dummy, fac_dummy2], axis=1)

In [9]:
X_train = pd.concat([train_dummy, train_data['beds'], train_data['bathrooms'], train_data['longitude'], train_data['latitude'], train_data['surface']], axis=1)
Y_train = train_data['log_price']

X_test = pd.concat([test_dummy, test_data['beds'], test_data['bathrooms'], test_data['longitude'], test_data['latitude'], test_data['surface']], axis=1)
Y_test = test_data['log_price']

In [10]:
X_train.shape

(2313, 187)

In [11]:
## checking for NaNs (Found 395 rows with missing value for Surface Area)
#for col in X_train:
    #print(col, ': ', X_train[col].isnull().sum())

print(Y_train.isnull().sum())

#for col in X_test:
    #print(col, ': ', X_test[col].isnull().sum())

print(Y_test.isnull().sum())

0
0


In [12]:
## to accommodate linear regression, where surface == NaN we will set value to average of surface
X_train['surface'] = X_train['surface'].fillna((X_train['surface'].mean()))

X_test['surface'] = X_test['surface'].fillna((X_test['surface'].mean()))

## train/test your model

In [13]:
reg = LinearRegression().fit(X_train, Y_train)
# predictions for training data
make_predictions(train_data, X_train, reg)

## Compute MdAPE: Median absolute percentage error (less sensitive to outliers than MAPE)

In [176]:
def MdAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return round(np.median(np.abs((y_true - y_pred) / y_true)) * 100, 2)


In [180]:
def MAPE(y_true,y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)

In [198]:
print('Training MdAPE: ', MdAPE(train_data['price'], train_data['predicted_price']), '%')
print('Training MAPE: ', MAPE(train_data['price'], train_data['predicted_price']), '%')

Training MdAPE:  12.48 %
Training MAPE:  18.82 %


In [38]:
#importance = reg.coef_
# summarize feature importance
#for i,v in enumerate(importance):
	#print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
#plt.bar([x for x in range(len(importance))], importance)
#plt.show()
#importance = -np.sort(-importance)
#top_50 = importance[0:50]
#for i, v in enumerate(top_50):
#	print('Feature: %0d, Score: %.5f' % (i,v))
#plt.bar([x for x in range(len(top_50))], top_50)
#plt.show()


array([8.43234626e+04, 8.85261028e+01, 3.62607542e+01, 3.05536039e+01,
       2.25146307e+01, 2.09095391e+01, 1.72802752e+01, 1.49353055e+01,
       1.47077879e+01, 1.44537257e+01, 1.42220228e+01, 1.35621198e+01,
       1.32955567e+01, 1.24123840e+01, 1.21799587e+01, 1.19182693e+01,
       1.17483988e+01, 1.16594365e+01, 1.14596985e+01, 1.00208943e+01,
       8.90765921e+00, 8.73268039e+00, 8.39044386e+00, 8.16649739e+00,
       8.03168097e+00, 7.83803454e+00, 7.73815482e+00, 7.61109270e+00,
       7.46758655e+00, 7.29789114e+00, 7.29029897e+00, 7.21546776e+00,
       7.05701712e+00, 7.01716953e+00, 6.95452686e+00, 6.87804884e+00,
       6.76782420e+00, 6.70969710e+00, 6.58781330e+00, 6.50078892e+00,
       6.27845787e+00, 6.18588129e+00, 5.85890063e+00, 5.75570847e+00,
       5.65880117e+00, 5.60787511e+00, 5.56431956e+00, 5.48657167e+00,
       5.42165800e+00, 5.36919121e+00, 5.32321994e+00, 5.27190092e+00,
       5.24135887e+00, 5.09642177e+00, 5.07163804e+00, 5.02755989e+00,
      